TALLER 3 - DETECIÓN DE ROSTRO CON IMAGENES 

In [1]:
import cv2
import customtkinter as ctk
from PIL import Image, ImageTk
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2

# Lista de imágenes y el índice actual
image_paths = [
    "becky.jpeg",   # Asegúrate de que las imágenes estén en el directorio correcto
    "descarga.jpeg",
    "sarocha.jpeg"
]
image_index = 0

# Variable para almacenar la imagen cargada
current_image = None

# Cargar el modelo de MediaPipe
with open(r"C:\Users\charl\Desktop\VA_environment\2. Talleres\taller3\Taller3Files\face_landmarker_v2_with_blendshapes.task", "rb") as model_file:
    model_data = model_file.read()

base_options = python.BaseOptions(model_asset_buffer=model_data)
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
    num_faces=1
)
detector = vision.FaceLandmarker.create_from_options(options)

# Función para dibujar los puntos faciales en la imagen
def draw_landmarks_on_image(rgb_image, detection_result):
    face_landmarks_list = detection_result.face_landmarks
    annotated_image = np.copy(rgb_image)

    for face_landmarks in face_landmarks_list:
        face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        face_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
        ])

        mp.solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style()
        )
        mp.solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_contours_style()
        )
        mp.solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_iris_connections_style()
        )

    return annotated_image

# Función para mostrar la imagen sin detección
def mostrar_imagen():
    global current_image
    if 0 <= image_index < len(image_paths):
        image_path = image_paths[image_index]
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error al cargar la imagen: {image_path}")
            return

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        current_image = image  # Almacena la imagen en BGR para la detección
        img = Image.fromarray(image_rgb)
        imgtk = ImageTk.PhotoImage(image=img)
        video_frame.imgtk = imgtk
        video_frame.configure(image=imgtk)

# Función para avanzar a la siguiente imagen
def siguiente_imagen():
    global image_index
    image_index += 1
    if image_index >= len(image_paths):
        image_index = 0  # Reinicia al principio después de la última imagen
    mostrar_imagen()

# Función para realizar la detección facial
def detectar_rostro():
    if current_image is None:
        return  # No hacer nada si no hay una imagen cargada

    # Convertir la imagen a RGB para el detector
    image_rgb = cv2.cvtColor(current_image, cv2.COLOR_BGR2RGB)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
    
    # Realizar la detección
    detection_result = detector.detect(image)

    # Dibujar los puntos faciales en la imagen
    annotated_image = draw_landmarks_on_image(image_rgb, detection_result)

    # Mostrar la imagen anotada
    img = Image.fromarray(annotated_image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_frame.imgtk = imgtk
    video_frame.configure(image=imgtk)

# Crear la ventana principal
ctk.set_appearance_mode("system")  # Modo de apariencia de la interfaz
window = ctk.CTk()
window.title("Detección Facial con OpenCV y MediaPipe")

# Crear un marco de imagen en la ventana
video_frame = ctk.CTkLabel(window)
video_frame.pack()

# Botones de la interfaz
button_frame = ctk.CTkFrame(window)
button_frame.pack(pady=10)

iniciar_button = ctk.CTkButton(button_frame, text="Iniciar", command=mostrar_imagen)
iniciar_button.grid(row=0, column=0, padx=5)

siguiente_button = ctk.CTkButton(button_frame, text="Siguiente", command=siguiente_imagen)
siguiente_button.grid(row=0, column=1, padx=5)

detectar_button = ctk.CTkButton(button_frame, text="Detectar", command=detectar_rostro)
detectar_button.grid(row=0, column=2, padx=5)

finalizar_button = ctk.CTkButton(button_frame, text="Finalizar", command=window.destroy)
finalizar_button.grid(row=0, column=3, padx=5)

# Ejecutar la ventana de Tkinter
window.mainloop()

Error al cargar la imagen: becky.jpeg


TALLER 3 - DETECIÓN DE MANOS CON IMAGENES

In [3]:
import cv2
import customtkinter as ctk
from PIL import Image, ImageTk
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2

# Lista de imágenes y el índice actual
image_paths = [
    "Manos 1.jpeg",   # Asegúrate de que las imágenes estén en el directorio correcto
    "Manos 2.jpeg",
    "Manos 3.1.jpeg"
]
image_index = 0

# Variable para almacenar la imagen cargada
current_image = None

# Cargar el modelo de MediaPipe para detección de manos
with open("hand_landmarker.task", "rb") as model_file:  # Cambia al archivo de modelo de manos
    model_data = model_file.read()

base_options = python.BaseOptions(model_asset_buffer=model_data)
options = vision.HandLandmarkerOptions(
    base_options=base_options,
    num_hands=2  # Número máximo de manos a detectar
)
detector = vision.HandLandmarker.create_from_options(options)

# Función para dibujar los puntos de referencia de la mano en la imagen
def draw_hand_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    annotated_image = np.copy(rgb_image)

    for hand_landmarks in hand_landmarks_list:
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])

        # Dibuja los puntos clave y las conexiones de las manos
        mp.solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=hand_landmarks_proto,
            connections=mp.solutions.hands.HAND_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_hand_connections_style()
        )

    return annotated_image

# Función para mostrar la imagen sin detección
def mostrar_imagen():
    global current_image
    if 0 <= image_index < len(image_paths):
        image_path = image_paths[image_index]
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error al cargar la imagen: {image_path}")
            return

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        current_image = image  # Almacena la imagen en BGR para la detección
        img = Image.fromarray(image_rgb)
        imgtk = ImageTk.PhotoImage(image=img)
        video_frame.imgtk = imgtk
        video_frame.configure(image=imgtk)

# Función para avanzar a la siguiente imagen
def siguiente_imagen():
    global image_index
    image_index += 1
    if image_index >= len(image_paths):
        image_index = 0  # Reinicia al principio después de la última imagen
    mostrar_imagen()

# Función para realizar la detección de manos
def detectar_manos():
    if current_image is None:
        return  # No hacer nada si no hay una imagen cargada

    # Convertir la imagen a RGB para el detector
    image_rgb = cv2.cvtColor(current_image, cv2.COLOR_BGR2RGB)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
    
    # Realizar la detección
    detection_result = detector.detect(image)

    # Dibujar los puntos de referencia de la mano en la imagen
    annotated_image = draw_hand_landmarks_on_image(image_rgb, detection_result)

    # Mostrar la imagen anotada
    img = Image.fromarray(annotated_image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_frame.imgtk = imgtk
    video_frame.configure(image=imgtk)

# Crear la ventana principal
ctk.set_appearance_mode("light")  # Modo de apariencia de la interfaz
window = ctk.CTk()
window.title("Detección de Manos con OpenCV y MediaPipe")

# Crear un marco de imagen en la ventana
video_frame = ctk.CTkLabel(window)
video_frame.pack()

# Botones de la interfaz
button_frame = ctk.CTkFrame(window)
button_frame.pack(pady=10)

iniciar_button = ctk.CTkButton(button_frame, text="Iniciar", command=mostrar_imagen)
iniciar_button.grid(row=0, column=0, padx=5)

siguiente_button = ctk.CTkButton(button_frame, text="Siguiente", command=siguiente_imagen)
siguiente_button.grid(row=0, column=1, padx=5)

detectar_button = ctk.CTkButton(button_frame, text="Detectar", command=detectar_manos)
detectar_button.grid(row=0, column=2, padx=5)

finalizar_button = ctk.CTkButton(button_frame, text="Finalizar", command=window.destroy)
finalizar_button.grid(row=0, column=3, padx=5)

# Ejecutar la ventana de Tkinter
window.mainloop()

FileNotFoundError: [Errno 2] No such file or directory: 'hand_landmarker.task'

TALLER 3 - TIEMPO REAL DE ROSTRO Y MANOS

In [ ]:
# Importación de librerías
from PIL import Image
from PIL import ImageTk
import cv2
import imutils
import customtkinter
import numpy as np
import math
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions

# Variables globales
cap = None
det = False

# Funciones de la aplicación
def iniciar():
    global cap
    cap = cv2.VideoCapture(0)
    visualizar()

def visualizar():
    global cap
    global det
    if cap is not None:
        ret, frame = cap.read()
        if ret:
            if det:
                frame = modificar(frame)
            frame = imutils.resize(frame, width=640)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            im = Image.fromarray(frame)
            img = customtkinter.CTkImage(light_image=im, size=(frame.shape[1], frame.shape[0]))
            lblVideo.configure(image=img)
            lblVideo.image = img
            lblVideo.after(10, visualizar)
        else:
            lblVideo.image = ""
            cap.release()

def detectar():
    global det
    det = not det

def finalizar():
    global cap
    global det
    det = False
    cap.release()  # Libera la cámara
    root.quit()    # Detiene el bucle de eventos de Tkinter
    root.destroy() # Cierra la ventana de la interfaz

def modificar(frame):
    proc_frame = detectorFaceAndHand(frame)
    return proc_frame

# Funciones de detección
from mediapipe.framework.formats import landmark_pb2

# Configuración de visualización
MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)

def detectorFaceAndHand(img):
    global detector, hands
    height, width, _ = img.shape
    annotated_image = img.copy()

    # Detección de rostro
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
    detection_result = detector.detect(image)
    annotated_image = visualize(annotated_image, detection_result)

    # Detección de manos
    results_hands = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            # Dibujar los puntos de las manos
            mp.solutions.drawing_utils.draw_landmarks(
                annotated_image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS,
                mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                mp.solutions.drawing_styles.get_default_hand_connections_style()
            )

            # Obtener las coordenadas de la muñeca para determinar si es derecha o izquierda
            wrist = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.WRIST]
            hand_x = wrist.x

            # Determinar si la mano es izquierda o derecha
            hand_label = "Derecha" if hand_x > 0.5 else "Izquierda"

            # Calcular la posición para mostrar la etiqueta en la imagen
            text_x = int(hand_x * width)
            text_y = int(wrist.y * height) - 10  # Coloca la etiqueta encima de la muñeca

            # Dibujar la etiqueta en la imagen
            cv2.putText(annotated_image, hand_label,
                        (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                        1, (255, 0, 0), 2, cv2.LINE_AA)

    return annotated_image

def visualize(image, detection_result) -> np.ndarray:
    """Dibuja cuadros delimitadores y puntos clave en la imagen de entrada."""
    height, width, _ = image.shape

    for detection in detection_result.detections:
        # Dibujar el cuadro delimitador del rostro
        bbox = detection.bounding_box
        start_point = (bbox.origin_x, bbox.origin_y)
        end_point = (bbox.origin_x + bbox.width, bbox.origin_y + bbox.height)
        cv2.rectangle(image, start_point, end_point, TEXT_COLOR, 3)

        # Dibujar puntos clave
        for keypoint in detection.keypoints:
            keypoint_px = _normalized_to_pixel_coordinates(keypoint.x, keypoint.y, width, height)
            cv2.circle(image, keypoint_px, 2, (0, 255, 0), -1)

    return image

def _normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
    """Convierte las coordenadas normalizadas a píxeles."""
    x_px = min(math.floor(normalized_x * image_width), image_width - 1)
    y_px = min(math.floor(normalized_y * image_height), image_height - 1)
    return x_px, y_px

# Configuración de detección
model_file = open(r'C:\Users\charl\Desktop\VA_environment\2. Talleres\taller3\Taller3Files\blaze_face_short_range.tflite', "rb")
model_data = model_file.read()
model_file.close()
base_options = python.BaseOptions(model_asset_buffer=model_data)
options = vision.FaceDetectorOptions(base_options=base_options)
detector = vision.FaceDetector.create_from_options(options)

# Configuración de MediaPipe para manos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Interfaz de usuario
root = customtkinter.CTk()

btnIniciar = customtkinter.CTkButton(root, text="Iniciar", width=45, command=iniciar)
btnIniciar.grid(column=0, row=0, padx=5, pady=5)

btnFinalizar = customtkinter.CTkButton(root, text="Finalizar", width=45, command=finalizar)
btnFinalizar.grid(column=1, row=0, padx=5, pady=5)

btnMediaPipe = customtkinter.CTkButton(root, text="Detectar", width=45, command=detectar)
btnMediaPipe.grid(column=2, row=0, padx=5, pady=5)

lblVideo = customtkinter.CTkLabel(root, text="")
lblVideo.grid(column=0, row=1, columnspan=3)

root.resizable(width=False, height=False)
root.mainloop()

TALLER 3 -  TIEMPO REAL DE ROSTRO 

In [4]:
# Importación de librerías
from PIL import Image
from PIL import ImageTk
import cv2
import imutils
import customtkinter
import numpy as np
import math
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Variables globales
cap = None
det = False

# Funciones de la aplicación
def iniciar():
    global cap
    cap = cv2.VideoCapture(0)
    visualizar()

def visualizar():
    global cap
    global det
    if cap is not None:
        ret, frame = cap.read()
        if ret:
            if det:
                frame = modificar(frame)
            frame = imutils.resize(frame, width=640)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            im = Image.fromarray(frame)
            img = customtkinter.CTkImage(light_image=im, size=(frame.shape[1], frame.shape[0]))
            lblVideo.configure(image=img)
            lblVideo.image = img
            lblVideo.after(10, visualizar)
        else:
            lblVideo.image = ""
            cap.release()

def detectar():
    global det
    det = not det

def finalizar():
    global cap
    global det
    det = False
    cap.release()  # Libera la cámara
    root.quit()    # Detiene el bucle de eventos de Tkinter
    root.destroy() # Cierra la ventana de la interfaz

def modificar(frame):
    proc_frame = detectorFace(frame)
    return proc_frame

# Función de detección de rostro
def detectorFace(img):
    height, width, _ = img.shape
    annotated_image = img.copy()

    # Detección de rostro
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
    detection_result = detector.detect(image)
    annotated_image = visualize(annotated_image, detection_result)

    return annotated_image

def visualize(image, detection_result) -> np.ndarray:
    """Dibuja cuadros delimitadores y puntos clave en la imagen de entrada."""
    height, width, _ = image.shape

    for detection in detection_result.detections:
        # Dibujar el cuadro delimitador del rostro
        bbox = detection.bounding_box
        start_point = (bbox.origin_x, bbox.origin_y)
        end_point = (bbox.origin_x + bbox.width, bbox.origin_y + bbox.height)
        cv2.rectangle(image, start_point, end_point, (255, 0, 0), 3)

        # Dibujar puntos clave
        for keypoint in detection.keypoints:
            keypoint_px = _normalized_to_pixel_coordinates(keypoint.x, keypoint.y, width, height)
            cv2.circle(image, keypoint_px, 2, (0, 255, 0), -1)

    return image

def _normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
    """Convierte las coordenadas normalizadas a píxeles."""
    x_px = min(math.floor(normalized_x * image_width), image_width - 1)
    y_px = min(math.floor(normalized_y * image_height), image_height - 1)
    return x_px, y_px

# Configuración de detección de rostro
model_file = open(r'C:\Users\charl\Desktop\VA_environment\2. Talleres\taller3\Taller3Files\blaze_face_short_range.tflite', "rb")
model_data = model_file.read()
model_file.close()
base_options = python.BaseOptions(model_asset_buffer=model_data)
options = vision.FaceDetectorOptions(base_options=base_options)
detector = vision.FaceDetector.create_from_options(options)

# Interfaz de usuario
root = customtkinter.CTk()

btnIniciar = customtkinter.CTkButton(root, text="Iniciar", width=45, command=iniciar)
btnIniciar.grid(column=0, row=0, padx=5, pady=5)

btnFinalizar = customtkinter.CTkButton(root, text="Finalizar", width=45, command=finalizar)
btnFinalizar.grid(column=1, row=0, padx=5, pady=5)

btnMediaPipe = customtkinter.CTkButton(root, text="Detectar", width=45, command=detectar)
btnMediaPipe.grid(column=2, row=0, padx=5, pady=5)

lblVideo = customtkinter.CTkLabel(root, text="")
lblVideo.grid(column=0, row=1, columnspan=3)

root.resizable(width=False, height=False)
root.mainloop()

TALLER 3 - TIEMPO REAL DE MANOS 

In [ ]:
# Importación de librerías
from PIL import Image
from PIL import ImageTk
import cv2
import imutils
import customtkinter
import numpy as np
import mediapipe as mp

# Variables globales
cap = None
det = False

# Funciones de la aplicación
def iniciar():
    global cap
    cap = cv2.VideoCapture(0)
    visualizar()

def visualizar():
    global cap
    global det
    if cap is not None:
        ret, frame = cap.read()
        if ret:
            if det:
                frame = modificar(frame)
            frame = imutils.resize(frame, width=640)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            im = Image.fromarray(frame)
            img = customtkinter.CTkImage(light_image=im, size=(frame.shape[1], frame.shape[0]))
            lblVideo.configure(image=img)
            lblVideo.image = img
            lblVideo.after(10, visualizar)
        else:
            lblVideo.image = ""
            cap.release()

def detectar():
    global det
    det = not det

def finalizar():
    global cap
    global det
    det = False
    cap.release()  # Libera la cámara
    root.quit()    # Detiene el bucle de eventos de Tkinter
    root.destroy() # Cierra la ventana de la interfaz

def modificar(frame):
    proc_frame = detectorHand(frame)
    return proc_frame

# Función de detección de manos
def detectorHand(img):
    height, width, _ = img.shape
    annotated_image = img.copy()

    # Detección de manos
    results_hands = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            # Dibujar los puntos de las manos
            mp.solutions.drawing_utils.draw_landmarks(
                annotated_image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS,
                mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                mp.solutions.drawing_styles.get_default_hand_connections_style()
            )

            # Obtener las coordenadas de la muñeca para determinar si es derecha o izquierda
            wrist = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.WRIST]
            hand_x = wrist.x

            # Determinar si la mano es izquierda o derecha
            hand_label = "Derecha" if hand_x > 0.5 else "Izquierda"

            # Calcular la posición para mostrar la etiqueta en la imagen
            text_x = int(hand_x * width)
            text_y = int(wrist.y * height) - 10  # Coloca la etiqueta encima de la muñeca

            # Dibujar la etiqueta en la imagen
            cv2.putText(annotated_image, hand_label,
                        (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                        1, (255, 0, 0), 2, cv2.LINE_AA)

    return annotated_image

# Configuración de MediaPipe para manos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Interfaz de usuario
root = customtkinter.CTk()

btnIniciar = customtkinter.CTkButton(root, text="Iniciar", width=45, command=iniciar)
btnIniciar.grid(column=0, row=0, padx=5, pady=5)

btnFinalizar = customtkinter.CTkButton(root, text="Finalizar", width=45, command=finalizar)
btnFinalizar.grid(column=1, row=0, padx=5, pady=5)

btnMediaPipe = customtkinter.CTkButton(root, text="Detectar", width=45, command=detectar)
btnMediaPipe.grid(column=2, row=0, padx=5, pady=5)

lblVideo = customtkinter.CTkLabel(root, text="")
lblVideo.grid(column=0, row=1, columnspan=3)

root.resizable(width=False, height=False)
root.mainloop()

c:\Users\Jacke\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
